In [37]:
import pandas as pd
import numpy as np
import os
import time
import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
### YOUR CODE HERE
from tensorflow.keras.regularizers import Regularizer
# Figure out how to import regularizers
import tensorflow.keras.utils as ku 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
%cd /content/drive/MyDrive/data/

/content/drive/MyDrive/data


In [39]:
#The dataset (P.S. Look, it is TSV)
data=pd.read_csv('sad.csv')

# removing null values to avoid errors 
data.dropna(inplace = True) 
  
# converting to string data type
data["Title"]= data["Title"].astype(str)
  
# slicing till 2nd last element
data["lyric"]= data["Title"].str.slice(10,)
  
# display
data.head()

# data.album.drop_duplicates().tolist()
data.head()

,Title,lyric
0,[00:19.48]Started a search to no avail,Started a search to no avail
1,[00:23.18]a light that shines behind the veil ...,a light that shines behind the veil trying to ...
2,[00:30.02]and all around us everywhere,and all around us everywhere
3,[00:34.30]is all that we could ever share if o...,is all that we could ever share if only we cou...
4,[00:40.75]feel there's truth that's beyond me,feel there's truth that's beyond me


In [40]:
data.lyric.drop_duplicates().tolist()

['Started a search to no avail',
 'a light that shines behind the veil trying to find it',
 'and all around us everywhere',
 'is all that we could ever share if only we could see it',
 "feel there's truth that's beyond me",
 'life ever changing weaving destiny',
 '',
 "And it feels like i'm flying above you",
 "dream that i'm dying to find the truth",
 'seems like your trying to bring me down',
 'back down to earth back down to earth',
 'Layers of dust and yesterdays',
 "shadows fading in the haze of what i couldn't say",
 'and though i said my hands were tied',
 "times have changed and now i find i'm free for the first time",
 'feel so close to everything now',
 'strange how life makes sense in time now',
 'Something always brings me back to you.',
 'It never takes too long.',
 'No matter what I say or do',
 "I'll still feel you here 'til the moment I'm gone.",
 'You hold me without touch.',
 'You keep me without chains.',
 'I never wanted anything so much than to drown in your love',

In [41]:
data.shape

(4857, 2)

In [42]:
IND = data.lyric.drop_duplicates().tolist()
#Move in the index range
CHORUSES=[]
for i in IND:
    CHORUSES.append(data.lyric.tolist())

In [43]:
CHORUSES=np.array(data.lyric.tolist())
CHORUSES=CHORUSES.tolist()
#Write everything in a single line 
TERM=''
for c in range(len(CHORUSES)):
    
    TERM= TERM+ CHORUSES[c]+ ' \n '

In [44]:
TERM

'Started a search to no avail \n a light that shines behind the veil trying to find it \n and all around us everywhere \n is all that we could ever share if only we could see it \n feel there\'s truth that\'s beyond me \n life ever changing weaving destiny \n  \n And it feels like i\'m flying above you \n dream that i\'m dying to find the truth \n seems like your trying to bring me down \n back down to earth back down to earth \n  \n Layers of dust and yesterdays \n shadows fading in the haze of what i couldn\'t say \n and though i said my hands were tied \n times have changed and now i find i\'m free for the first time \n feel so close to everything now \n strange how life makes sense in time now \n  \n And it feels like i\'m flying above you \n dream that i\'m dying to find the truth \n seems like your trying to bring me down \n back down to earth back down to earth \n back down to earth back down to earth \n  \n Something always brings me back to you. \n It never takes too long. \n 

In [45]:
print("The unique characters:",sorted(set(TERM)))

The unique characters: ['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', '[', '\\', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~', '—', '‘', '’', '…', '人', '作', '兔', '刀', '制', '危', '墨', '子', '宝', '很', '拿', '林', '泉', '的', '菜', '险', '（', '）']


In [46]:
to_remove = ['[', '\\', ']', '_', '{', '}','~', '§', '…','≡', '了', '人', '作', '制', '卐', '我', '朝', '王', '语', '\ufeff','(', ')','*', '（', '）', '‘', '’', '′', '＇', "'", "'", '.']
for symbol in to_remove:
    TERM = TERM.replace(symbol,"")

print("The unique characters:",sorted(set(TERM)))

The unique characters: ['\n', ' ', '!', '"', '&', ',', '-', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '兔', '刀', '危', '墨', '子', '宝', '很', '拿', '林', '泉', '的', '菜', '险']


In [47]:
#Write it into a file
f = open("sadchorus.txt", "a")
f.write(TERM)
f.close()

In [48]:
text = open('sadchorus.txt', 'rb').read().decode(encoding='utf-8')

In [49]:
text[0:5000]

'Started a search to no avail \n a light that shines behind the veil trying to find it \n and all around us everywhere \n is all that we could ever share if only we could see it \n feel theres truth thats beyond me \n life ever changing weaving destiny \n  \n And it feels like im flying above you \n dream that im dying to find the truth \n seems like your trying to bring me down \n back down to earth back down to earth \n  \n Layers of dust and yesterdays \n shadows fading in the haze of what i couldnt say \n and though i said my hands were tied \n times have changed and now i find im free for the first time \n feel so close to everything now \n strange how life makes sense in time now \n  \n And it feels like im flying above you \n dream that im dying to find the truth \n seems like your trying to bring me down \n back down to earth back down to earth \n back down to earth back down to earth \n  \n Something always brings me back to you \n It never takes too long \n No matter what I s

In [50]:
# Splitting the string into sentences, while converting whole data into lowercase.
corpus = text.lower().split("\n")
# To make sure no sentence appears twice in our corpus, we use set. Otherwise, it will make the model biased.
corpus = list(set(corpus))
print(corpus)

[' so maybe i seem loose to you ', ' clouds are coming down the chimney ', ' after all ', ' you bought me shoes ', ' and his heart aches, ', ' oh my beds so cold at night and i miss you more each day ', ' and an old tooth i believe was a bears ', ' what a rainy ending give to a perfect day ', ' coffee laced intoxicating on her lips ', ' but the answers arent the same ', ' i would keep myself ', ' the little holes in your vanes ', ' in your eyes ', ' tell me baby, do you recognize me ', ' why you were left alone ', ' i konw its hard babe to sleep at night ', ' i lead them over to your house ', ' say love you you loving me yeah ', ' in the middle of saying something ', ' from the ice inside your soul ', ' risking it all in a glance ', ' that night you whispered like a ghost and you looked so shaken ', ' if you came back as the deep sea, ', ' i wonder where you are tonight ', ' cause im always thinking, thinking about where you are ', ' i keep my distance ', ' and where will i go ', ' whe

In [51]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [52]:
tokenizer.word_index

{'i': 1,
 'you': 2,
 'the': 3,
 'and': 4,
 'to': 5,
 'a': 6,
 'me': 7,
 'it': 8,
 'my': 9,
 'in': 10,
 'your': 11,
 'that': 12,
 'of': 13,
 'im': 14,
 'all': 15,
 'is': 16,
 'be': 17,
 'but': 18,
 'know': 19,
 'on': 20,
 'its': 21,
 'for': 22,
 'so': 23,
 'we': 24,
 'just': 25,
 'dont': 26,
 'if': 27,
 'when': 28,
 'like': 29,
 'now': 30,
 'love': 31,
 'never': 32,
 'was': 33,
 'what': 34,
 'see': 35,
 'not': 36,
 'are': 37,
 'how': 38,
 'will': 39,
 'with': 40,
 'go': 41,
 'this': 42,
 'do': 43,
 'have': 44,
 'down': 45,
 'back': 46,
 'cause': 47,
 'time': 48,
 'still': 49,
 'youre': 50,
 'here': 51,
 'oh': 52,
 'up': 53,
 'away': 54,
 'can': 55,
 'no': 56,
 'there': 57,
 'let': 58,
 'want': 59,
 'were': 60,
 'one': 61,
 'could': 62,
 'ill': 63,
 'make': 64,
 'from': 65,
 'she': 66,
 'cant': 67,
 'say': 68,
 'come': 69,
 'her': 70,
 'at': 71,
 'ever': 72,
 'heart': 73,
 'through': 74,
 'ive': 75,
 'as': 76,
 'again': 77,
 'la': 78,
 'life': 79,
 'would': 80,
 'out': 81,
 'too': 82,
 '

In [53]:
# create input sequences using list of tokens
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [54]:
input_sequences

[[23, 121],
 [23, 121, 1],
 [23, 121, 1, 290],
 [23, 121, 1, 290, 769],
 [23, 121, 1, 290, 769, 5],
 [23, 121, 1, 290, 769, 5, 2],
 [616, 37],
 [616, 37, 260],
 [616, 37, 260, 45],
 [616, 37, 260, 45, 3],
 [616, 37, 260, 45, 3, 1106],
 [209, 15],
 [2, 1107],
 [2, 1107, 7],
 [2, 1107, 7, 617],
 [4, 318],
 [4, 318, 73],
 [4, 318, 73, 1108],
 [52, 9],
 [52, 9, 1109],
 [52, 9, 1109, 23],
 [52, 9, 1109, 23, 112],
 [52, 9, 1109, 23, 112, 71],
 [52, 9, 1109, 23, 112, 71, 89],
 [52, 9, 1109, 23, 112, 71, 89, 4],
 [52, 9, 1109, 23, 112, 71, 89, 4, 1],
 [52, 9, 1109, 23, 112, 71, 89, 4, 1, 249],
 [52, 9, 1109, 23, 112, 71, 89, 4, 1, 249, 2],
 [52, 9, 1109, 23, 112, 71, 89, 4, 1, 249, 2, 140],
 [52, 9, 1109, 23, 112, 71, 89, 4, 1, 249, 2, 140, 388],
 [52, 9, 1109, 23, 112, 71, 89, 4, 1, 249, 2, 140, 388, 96],
 [4, 197],
 [4, 197, 174],
 [4, 197, 174, 770],
 [4, 197, 174, 770, 1],
 [4, 197, 174, 770, 1, 185],
 [4, 197, 174, 770, 1, 185, 33],
 [4, 197, 174, 770, 1, 185, 33, 6],
 [4, 197, 174, 770, 

In [55]:
corpus[0]

' so maybe i seem loose to you '

In [56]:
tokenizer.word_index["still"]

49

In [57]:
# pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences,
                       maxlen = max_sequence_len, padding='pre'))

In [58]:
input_sequences

array([[   0,    0,    0, ...,    0,   23,  121],
       [   0,    0,    0, ...,   23,  121,    1],
       [   0,    0,    0, ...,  121,    1,  290],
       ...,
       [   0,    0,    0, ...,    3, 1097,  767],
       [   0,    0,    0, ..., 1097,  767,    3],
       [   0,    0,    0, ...,  767,    3,  112]], dtype=int32)

In [59]:
input_sequences.shape

(15571, 16)

In [60]:
tokenizer.word_index['universe']

649

In [61]:
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [62]:
label.shape

(15571, 2232)

In [63]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))  #(# Your Embedding Layer)
model.add(Bidirectional(LSTM(150, return_sequences=True)))  #(# An LSTM Layer)
model.add(Dropout(0.2))  #(# A dropout layer)
model.add(LSTM(100))  #(# Another LSTM Layer)
model.add(Dense(total_words/2, activation='relu'))  #(# A Dense Layer including regularizers)
model.add(Dense(total_words, activation='softmax'))  #(# A Dense Layer)
# Pick an optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')  #(# Pick a loss function and an optimizer)
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 15, 100)           223200    
                                                                 
 bidirectional_1 (Bidirectio  (None, 15, 300)          301200    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 15, 300)           0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               160400    
                                                                 
 dense_2 (Dense)             (None, 1116)              112716    
                                                                 
 dense_3 (Dense)             (None, 2232)              2493144   
                                                      

In [64]:
model.fit(predictors, label, epochs= 50, verbose=1)

Epoch 1/50
487/487 [==============================] - 9s 11ms/step - loss: 6.1661 - accuracy: 0.0419
Epoch 2/50
487/487 [==============================] - 5s 11ms/step - loss: 5.6439 - accuracy: 0.0599
Epoch 3/50
487/487 [==============================] - 5s 10ms/step - loss: 5.3439 - accuracy: 0.0809
Epoch 4/50
487/487 [==============================] - 5s 10ms/step - loss: 5.1327 - accuracy: 0.0965
Epoch 5/50
487/487 [==============================] - 5s 10ms/step - loss: 4.9667 - accuracy: 0.1055
Epoch 6/50
487/487 [==============================] - 5s 11ms/step - loss: 4.7949 - accuracy: 0.1174
Epoch 7/50
487/487 [==============================] - 5s 10ms/step - loss: 4.6451 - accuracy: 0.1250
Epoch 8/50
487/487 [==============================] - 5s 11ms/step - loss: 4.5062 - accuracy: 0.1352
Epoch 9/50
487/487 [==============================] - 6s 12ms/step - loss: 4.3816 - accuracy: 0.1453
Epoch 10/50
487/487 [==============================] - 5s 10ms/step - loss: 4.2595 - accura

In [65]:
from keras.models import load_model


In [69]:
model.save('sad_model.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
# model = load_model('my_model.h5')

In [66]:
def make_lyrics(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
        predicted = model.predict(token_list, verbose=0)
        classes = np.argmax(predicted,axis=1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == classes:
                output_word = word
                break
        seed_text += " " + output_word
    print(str(seed_text))

yhat_classes = model.predict_classes(X_test)

predict_x=model.predict(X_test) 

classes_x=np.argmax(predict_x,axis=1)

model.predict(x_test) > 0.5).astype("int32")

In [68]:
make_lyrics("sun", 10)

sun up in the eye died away to you come home
